In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from datetime import datetime

In [2]:
spark = SparkSession.builder.master("local").appName("ML").getOrCreate()

In [3]:
spark

In [4]:
Access_key_ID="*******************"
Secret_access_key="*********************************"
# Enable hadoop s3a settings
spark.sparkContext._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", \
                                     "com.amazonaws.auth.InstanceProfileCredentialsProvider,com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A")

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key",Access_key_ID)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key",Secret_access_key)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")

In [5]:
# Configure Redshift credentials and database information
redshift_jdbc_url = "jdbc:redshift://redshift.clxkpdwlmkpt.us-east-2.redshift.amazonaws.com:5439/dev"
redshift_user = "awsuser"
redshift_password = "AWS$user7"
redshift_database = "dev"
redshift_table = "bank"

In [6]:
# Reading data from Redshift 

df_reader = spark.read \
    .format("jdbc") \
    .option("url", redshift_jdbc_url) \
    .option("dbtable", redshift_table) \
    .option("user", redshift_user) \
    .option("password", redshift_password) \
    .option("fetchsize", "3000")

df = df_reader.load().limit(3000)

df.show()

+---+------------+--------+---------+--------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+------+
|age|         job| marital|education|default1|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+------------+--------+---------+--------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+------+
| 71|  management|  single| tertiary|     yes|  30398|     no|  no|  unknown|  2|  sep|     874|      24|   37|       6| success|    no|
| 39|    services|  single|  primary|     yes|  20995|    yes|  no|  unknown| 29|  apr|    4782|       2|  165|      14| unknown|   yes|
| 40|entrepreneur| married|  primary|     yes|  47421|    yes|  no|  unknown| 10|  nov|    3939|      10|  427|      15| failure|    no|
| 74| blue-collar|  single|  primary|     yes|  44915|     no|  no|telephone| 13|  apr|    1242|      42|  481|      18| unknown|    no|
| 22|entrepreneur|divorced|secondary|    

In [7]:
df.count()

3000

In [9]:
!pip install pandas

In [10]:
!pip install boto3

In [18]:
import pandas as pd
import boto3
from io import StringIO
new_df = df.toPandas()

In [19]:
new_df

,age,job,marital,education,default1,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,target
0,71,management,single,tertiary,yes,30398,no,no,unknown,2,sep,874,24,37,6,success,no
1,39,services,single,primary,yes,20995,yes,no,unknown,29,apr,4782,2,165,14,unknown,yes
2,40,entrepreneur,married,primary,yes,47421,yes,no,unknown,10,nov,3939,10,427,15,failure,no
3,74,blue-collar,single,primary,yes,44915,no,no,telephone,13,apr,1242,42,481,18,unknown,no
4,22,entrepreneur,divorced,secondary,yes,30806,no,yes,cellular,12,oct,1814,2,611,4,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,31,unemployed,married,secondary,no,3242,no,no,unknown,25,jun,292,2,-1,0,unknown,no
2996,59,technician,married,primary,no,3163,yes,no,unknown,29,may,240,2,-1,0,unknown,no
2997,50,unemployed,divorced,secondary,no,32,no,no,cellular,4,aug,282,2,-1,0,unknown,no
2998,47,admin.,married,secondary,no,310,yes,yes,telephone,14,aug,128,6,-1,0,unknown,no


In [20]:
# Loading Sample Data into S3 bucket

csv_buffer = StringIO()
new_df.to_csv(csv_buffer, index=False)

# create a Boto3 S3 client
s3 = boto3.client('s3')

# set the S3 bucket name and CSV file name
bucket_name = 'client-data-source'
csv_file_name = 'ml.csv'

# upload the CSV file to S3
s3.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key=csv_file_name)

{'ResponseMetadata': {'RequestId': 'MT0PTGKHGWJ6J5VS',
  'HostId': 'LkA2khF0chm2FYrl8EQYBryyUAwI7/OgUkD/rLCiOyuMPUTnMWetVVPWEND1mEUT9aVhK+Dx8SE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LkA2khF0chm2FYrl8EQYBryyUAwI7/OgUkD/rLCiOyuMPUTnMWetVVPWEND1mEUT9aVhK+Dx8SE=',
   'x-amz-request-id': 'MT0PTGKHGWJ6J5VS',
   'date': 'Sat, 18 Mar 2023 12:58:41 GMT',
   'x-amz-version-id': 'fziiMeXf23AYxyzhqEB2mciLc4QQWFMw',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9447239299a1b9ee510e1a4c65c6c92d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"9447239299a1b9ee510e1a4c65c6c92d"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'fziiMeXf23AYxyzhqEB2mciLc4QQWFMw'}